In [ ]:
import time
from stackapi import StackAPI
SITE = StackAPI('stackoverflow')

# https://api.stackexchange.com/docs/advanced-search
so_filter = "!-NKTjL10PP0gsCOfCLUV2tJ6bCzs1*0Tu"


In [ ]:
import re

# Extract code from SO answer
def extract_code(text):

    regex = r"<pre><code>(.|\n)*?<\/code><\/pre>"

    matches = re.finditer(regex, text, re.MULTILINE)
    code = []
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):  
            snippet = match.group(groupNum).replace("<pre>","").replace("<code>","").replace("</code>","").replace("</pre>","")
            code.append(snippet)
    return code

import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
# Create files:
question_title_file = open("sodata/title.txt","a", encoding="utf-8")
question_body_file = open("sodata/body.txt","a", encoding="utf-8")
answer_code_file = open("sodata/code.txt","a", encoding="utf-8")
meta_code_file = open("sodata/meta.txt","a", encoding="utf-8")

In [ ]:
tags = ["keras"] # "deep-learning;python","machine-learning;python",pandas

questions_processed = []

In [ ]:
## Get existing questions (in other tags)
open("sodata/keras/meta.txt","a", encoding="utf-8")
file = open("sodata/keras/meta.txt", 'r')
existing_meta = file.read()
for entry in existing_meta.split("\n"):        
    questions_processed.append((entry.split(",")[0]))

file.close()

In [ ]:
print(len(questions_processed))

In [ ]:

for tag in tags:
    
    print("Tag",tag)
    
    has_more = True

    page_number = 1

    while has_more :

        count = 0
        
        # Stack overflow bans IP if they receive >3 requests per second
        time.sleep(20)
        
        print("Page",page_number, "tag", tag)
        questions = SITE.fetch('search/advanced', page=page_number, pagesize=100, tagged=tag, sort='votes', accepted=True, filter=so_filter)

        print("number of questions", len(questions['items']))
        for question in questions['items']:
            
            count+=1
            if(count%250 == 0):
                print(count,"/",len(questions["items"]))
            
            if question["question_id"] in questions_processed:
                continue

            questions_processed.append(question["question_id"])

            accepted_answer = {}
            for answer in question["answers"]:
                if answer["is_accepted"]:
                    accepted_answer = answer

            if accepted_answer == {}:
                #print("Question ",question["question_id"],"has no accepted answer")
                continue

            title = question["title"]
            link = question["link"]
            body = cleanhtml(question["body"])
            code = extract_code(accepted_answer["body"])

            if len(code) == 0:
                #print("Question ",question["question_id"],"has no answer with code")
                continue

            question_title_file.write("%s\n" % title)
            question_body_file.write("%s\n" % repr(body))
            answer_code_file.write("%s\n" % repr("\n".join(code)))
            meta_code_file.write("%s\n" % (str(question["question_id"])+","+link)  )

        has_more = questions["has_more"]
        print("Page ended", "has more?",has_more)
        page_number += 1
            
print("End")
question_title_file.close()
question_body_file.close()
answer_code_file.close()
meta_code_file.close()
print("Finish")

In [ ]:
question_title_file.close()
question_body_file.close()
answer_code_file.close()
meta_code_file.close()

In [ ]:
questions["items"][0].keys()

In [ ]:
questions["page"]

In [ ]:
questions["has_more"]

In [ ]:
questions = SITE.fetch('search/advanced', page=1, pagesize=2, tagged='python;tensorflow', q='title:how', sort='votes', accepted=True, filter=so_filter)

In [ ]:
questions["items"][0]

In [ ]:
for q in questions["items"]:
    print(q["title"])

In [ ]:
questions = SITE.fetch('search/advanced', page=360, pagesize=100, tagged='python;tensorflow', q='title:how', sort='votes', accepted=True, filter=so_filter)


In [ ]:
questions["has_more"]